In [3]:
from transformers import pipeline, Pipeline

/Users/tharhtet/.local/share/virtualenvs/sample_fastapi-yZw_G79o/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import tensorflow as tf

tf.test.is_gpu_available()

2025-02-09 10:46:59.554704: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-09 10:46:59.554721: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


True

In [5]:
import torch

device_name = None
if torch.backends.mps.is_available():
    device_name = "mps"  # cuda
else:
    device_name = "cpu"

In [6]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16,
    device=device_name,
)

In [7]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
outputs = pipe(
    prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95
)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
As a rule of thumb, humans can eat about 200-300 calories per pound of body weight. However, individual weight and calorie needs can vary, so it's a good idea to monitor your calorie intake and adjust your diet accordingly. The exact amount of calories a human can consume in one sitting is not specified in the given text. It's best to consult with a healthcare professional for a personalized dietary plan.


In [13]:
sample = outputs[0]["generated_text"]
len(sample.split("<|assistant|>"))

2